In [1]:
import importlib
import local_support  

importlib.reload(local_support)


from local_support import *

Training for regression


In [2]:
cloud_dataset = np.load('datasets/train_subset_500k.npz')

In [3]:
X_train_set = cloud_dataset['X_train']
y_train_set = cloud_dataset['y_train']


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X_train_set, y_train_set, test_size=0.2, random_state=42)

# Kiểm tra kích thước của các tập dữ liệu
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")
train_set = (X_train, y_train)

X_train shape: (400000, 2)
X_test shape: (100000, 2)
y_train shape: (400000,)
y_test shape: (100000,)


In [5]:

best_regressor, best_params  = cross_validate(
    train_data=(X_train_set, y_train_set), 
    model_class=regressors['random_forest'], 
    param_grid=param_grids_regressor['random_forest'], 
    metric='neg_mean_squared_error',  # Use the correct scoring for MSE
    num_fold=5
)

Average R²: 0.7405967854469602
Average MSE: 0.04829978764332069
Best R²: 0.7417613523435197


In [6]:
folder_path = 'model_train/regressor'
file_path = os.path.join(folder_path, 'fill_nan_model_rf.pkl')
joblib.dump(best_regressor, file_path)

['model_train/regressor/fill_nan_model_rf.pkl']